In [3]:
import chess
import chess.engine
import chess.pgn

def analyze_game(game, engine, depth=15, show_move_cpl=False):
    # Extract information
    white_rating = game.headers.get("WhiteElo", "Unknown")
    black_rating = game.headers.get("BlackElo", "Unknown")
    opening = game.headers.get("Opening", "Unknown")
    variation = game.headers.get("Variation", "Unknown")

    cpl_white = []
    cpl_black = []

    board = game.board()

    for move in game.mainline_moves():
        # Analyze the position before the move
        best_info = engine.analyse(board, chess.engine.Limit(depth=depth))
        best_move_eval = best_info["score"].pov(board.turn).score(mate_score=100000)

        # Get the evaluation of the move played
        played_info = engine.analyse(board, chess.engine.Limit(depth=depth), root_moves=[move])
        played_eval = played_info["score"].pov(board.turn).score(mate_score=100000)

        # Compute centipawn loss
        if best_move_eval is not None and played_eval is not None:
            cpl = abs(best_move_eval - played_eval)
        else:
            cpl = 0

        # Assign CPL to the player who made the move
        if board.turn == chess.WHITE:
            cpl_white.append(cpl)
        else:
            cpl_black.append(cpl)

        # Make the move
        board.push(move)

    # Compute average CPL per player
    avg_cpl_white = sum(cpl_white) / len(cpl_white) if cpl_white else 0
    avg_cpl_black = sum(cpl_black) / len(cpl_black) if cpl_black else 0

    # Prepare results
    results = {
        "WhiteElo": white_rating,
        "BlackElo": black_rating,
        "Opening": opening,
        "Variation": variation,
        "CPL_White": cpl_white,
        "CPL_Black": cpl_black,
        "Average_CPL_White": avg_cpl_white,
        "Average_CPL_Black": avg_cpl_black
    }

    return results

def analyze_pgn_file(pgn_file_path, stockfish_path, depth=15, show_move_cpl=False):
    # Initialize the engine
    engine = chess.engine.SimpleEngine.popen_uci(stockfish_path)

    # Open the PGN file
    with open(pgn_file_path, 'r', encoding='utf-8') as pgn:
        game_number = 1
        while True:
            game = chess.pgn.read_game(pgn)
            if game is None:
                break
            print(f"Game {game_number}:")
            results = analyze_game(game, engine, depth, show_move_cpl)
            
            # Output the results
            print("White rating:", results["WhiteElo"])
            print("Black rating:", results["BlackElo"])
            print("Opening:", results["Opening"])
            print("Variation:", results["Variation"])
            
            if show_move_cpl:
                print("CPL per move:")
                move_number = 1
                for i in range(max(len(results["CPL_White"]), len(results["CPL_Black"]))):
                    if i < len(results["CPL_White"]):
                        print(f"Move {move_number} (White): {results['CPL_White'][i]}")
                        move_number += 1
                    if i < len(results["CPL_Black"]):
                        print(f"Move {move_number} (Black): {results['CPL_Black'][i]}")
                        move_number += 1
            
            print("Average CPL for White:", results["Average_CPL_White"])
            print("Average CPL for Black:", results["Average_CPL_Black"])
            print("\n-----------------------------\n")
            
            game_number += 1

    # Quit the engine
    engine.quit()

# Example usage:
stockfish_path = r"C:\Users\foivo\Downloads\stockfish-windows-x86-64-avx2\stockfish\stockfish-windows-x86-64-avx2.exe"
pgn_file_path = r"C:\Users\foivo\Documents\chess project\example2.pgn"
analyze_pgn_file(pgn_file_path, stockfish_path, depth=13, show_move_cpl=False)


Game 1:
White rating: 2755
Black rating: 2733
Opening: English
Variation: Caro-Kann defensive system
Average CPL for White: 18.720588235294116
Average CPL for Black: 23.850746268656717

-----------------------------

Game 2:
White rating: 2701
Black rating: 2632
Opening: English
Variation: four knights, Capablanca variation
Average CPL for White: 7.983333333333333
Average CPL for Black: 8.966666666666667

-----------------------------

